In [1]:
%%writefile C:\Users\Cflhxb\AppData\Local\anaconda3\Lib\CAMP.py
    
import datetime as dt
import pandas as pd
import numpy as np
import math as math 
import MathsUtilities as MUte
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy.optimize import curve_fit

CampConstants = {
    'k':-0.17,
    'VIThreshold':1.0,
    'TSThreshold':2.0,
    'SlopeFLNvsTS':1.1,
    'IntFLNvsTS':2.85,
    'PpMax':16,
    'PpMin':8,
}

colors = ['r','b','b','r']
fills = ['w','b','w','r']

def CalcdHS(Tt,Pp,HS,BasePhyllo,PhylloPpSens):
    """Calculate daily increase in Haun stage
    Args:
        Tt: Thermal time increment
        Pp: The current photoperiod
        HS: Current Haun Stage
        BasePhyllo: Base phyllochron at 16h Pp between 3 and 7 HS
        PhylloPpSens: Photo period sensitivity of phyllochron.  Relative increase in phyllochron as 8h Pp
    Returns:
        Daily increas in Haun Stage
    """
    HS = max(0,HS)
    StageFactor = np.interp(HS,[0,2,2.0001,7,7.0001,12],[0.75,0.75,1.0,1.0,1.4,1.4],False)
    PpFactor = 1 + PhylloPpSens * np.interp(Pp,[0,8,12,20],[1,1,0,0],False)
    Phyllochron = BasePhyllo*StageFactor*PpFactor
    return Tt/Phyllochron

def CalcPpResponse(Pp, Base, Max):
    """ Calculate value of a photoperiod sensitive variable between Max and Baee values
    Args:
        Pp: Photoperiod
        Base: value below 8h Pp
        Max: value above 16h Pp
        dPB: delta base phyllochrons
    Returns:
        variable value dependent on photoperiod and adjusted for dHS
    """
    if Pp <= CampConstants['PpMin']:
        return Base
    if (Pp > CampConstants['PpMin']) and (Pp < CampConstants['PpMax']):
        return (Base + (Max-Base) * (Pp-CampConstants['PpMin'])/(CampConstants['PpMax']-CampConstants['PpMin']))
    if (Pp >= CampConstants['PpMax']):
        return Max

def CalcBaseUpRegVrn1(Tt, dHS, BaseDVrn1):
    """ Calculate upregulation of base Vrn1
    
    Args:
        Tt: Thermal time increment
        dHS: Delta HaunStage
        BaseDVrn1: coeffociennt for Base Vrn1 expression
        
    Returns:
        delta BaseVrn1 representing the additional Vrn1 expression from base expression
    """
    if Tt < 0: 
        BaseDVrn1 = 0
    return BaseDVrn1 * dHS

def CalcColdUpRegVrn1(Tt,dHS, MaxApDev1, k):
    """ Upregulation of Vrn1 from cold.  Is additional to base vrn1
        BaseDVrn1 in seperate calculation otherwise te same as Brown etal 2013
    Args:
        Tt: Thermal time increment
        dHS: Delta Base Phyllochrons
        MaxApDev1: coefficient for Maximum upregulation of Vrn1
        k: The exponential coefficient determining temperature response
        
    Returns:
        delta ColdVrn1 representing the additional Vrn1 expression from cold upregulation
    """
    UdVrn1 = MaxApDev1 * np.exp(k*Tt)
    if (Tt < 20):
        return UdVrn1 * dHS
    else:
        return -5
        
def calcTSHS(FLN,IntFLNvsTSHS,SlopeFLNvsTS):
    """Haun stage timing of terminal spikelet 
       Inverts equation 5 from Brown etal 2013 FLN =  2.85 + 1.1*TSHS and converts it to base phyllochrons.
       Note the intercept differs, was typeo on publication
       Intercept has been made variable for as needs to be lower for some very fast varieties
    Args:
        FLN: The final leaf number
        IntFLNvsTSHS: The intercept of the regression between FLN and TSHS
    Returns:
        Estimation of number of base phyllochrons to terminal spikelet
    """
    return (FLN - IntFLNvsTSHS)/SlopeFLNvsTS

def CAMPmodel(Out, Day, Tt, Pp, Params, Consts, TtEmerge):
    """ The Cereal Anthesis Molecular Phenology model.
        Based on the ideas presented in Brown etal 2014 (Annals of Botany)
        Alterations made replacing Vrn4 notion with methalation of Vrn1, allowing Vrn3 to act before VI and changint the pattern for Vrn2 expression
        Other alterations to implement working code base
    Args:
        Out: "FLN" or None.  If "FLN" will only return estimated FLN else will return full dataframe with daily state variable values
        Day: List, 1:EndDay representing the timesteps in model run
        Tt: List of same length as Day, representing daily temperature
        Pp: List of same length as Day, representing daily photoperiod
        Params: Dict with genotype parameters fitted by CalcCultivarVrnCoeffs() 
        Consts: Dict with crop specific constants,
        TtEmerge: Thermaltime from sowing to emergence
    """
    # Set up Data structure and initialise values
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    DF = pd.DataFrame(index = [0], columns = ['Day','Tt','Pp'])
    IsGerminated = False
    IsEmerged = False
    IsMethalating = False
    IsVernalised = False
    IsReproductive = False
    IsAtFlagLeaf = False
    DF.loc[0,'Day'] = 0
    DF.loc[0,'Tt'] = Tt[0]
    DF.loc[0,'RelCold'] = 0
    DF.loc[0,'Pp'] = Pp[0]
    DF.loc[0,'fPP'] = 0
    DF.loc[0,'Stage'] = 'Germination'
    DF.loc[0,'IsGerminated'] =True
    DF.loc[0,'IsEmerged'] = False
    DF.loc[0,'IsVernalised'] = False
    DF.loc[0,'IsReproductive'] = False
    DF.loc[0,'IsAtFlagLeaf'] = False
    DF.loc[0,'HS'] = -TtEmerge/Params.BasePhyllo
    DF.loc[0,'dHS'] = 0
    DF.loc[0,'AccumTt'] = Tt[0]
    DF.loc[0,'VrnB'] = 0
    DF.loc[0,'dVrnB'] = 0
    DF.loc[0,'MaxApDev'] = 0
    DF.loc[0,'dMaxApDev'] = 0
    DF.loc[0,'Cold'] = 0
    DF.loc[0,'dCold'] = 0
    DF.loc[0,'ApDev'] = 0
    DF.loc[0,'MaxVrn2'] = 0
    DF.loc[0,'Vrn2'] = 0.0
    DF.loc[0,'AccumSPp'] = 0.0
    #DF.loc[0,'Vrn2ef'] = 0.0
    DF.loc[0,'Vrn1'] = 0.0
    DF.loc[0,'dVrn1'] = 0
    DF.loc[0,'Vrn3'] = 0.0
    DF.loc[0,'dVrn3'] = 0
    DF.loc[0,'VIHS'] = 0
    DF.loc[0,'TSHS'] = 0
    DF.loc[0,'FLN'] = Consts['IntFLNvsTS']
    
    d = 1
    
    # Model daily loop
    # ^^^^^^^^^^^^^^^^
    while (IsAtFlagLeaf == False) and (d < Day[-1]):
        ColdYesterday = 0
        ColdYesterday = DF.loc[d-1,'Cold']
        DF.loc[d,:] = DF.loc[d-1,:] #Carry yesterdays values over to today
        DF.loc[d,'Stage'] = np.nan
            
        DF.loc[d,'Day'] = d
        # Set daily environment variables
        DF.loc[d,'Tt'] = Tt[d]
        DF.loc[d,'Pp'] = Pp[d]
        #Zero set deltas
        DF.loc[d,'dHS'] = 0
        DF.loc[d,'dVrnB'] = 0
        DF.loc[d,'dMaxApDev'] = 0
        DF.loc[d,'dCold'] = 0
        DF.loc[d,'Vrn2'] = 0
        DF.loc[d,'dVrn1'] = 0
        DF.loc[d,'dVrn3'] = 0
        
                
        DF.loc[d,'AccumTt'] = DF.loc[d,'AccumTt'] + DF.loc[d,'Tt']
        
        PropnOfDay = 1.0
        if (DF.loc[d,'AccumTt'] > TtEmerge) and (IsEmerged==False):
            IsEmerged = True
            DF.loc[d,'Stage'] = 'Emergence'
            PropnOfDay = (DF.loc[d,'AccumTt'] - TtEmerge)/DF.loc[d,'Tt'] # Calculate fraction of emergence days Tt that is not used for emergence
        
        # Calculate daily Haun Stage changes
        # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        if IsEmerged==False: # Crop not yet emerged 
            EmergDurationFactor = 1
            if (DF.loc[d,'AccumTt'] > 90):  # Calculate EmergenceDurationFactor to slow accumulation of HS if emergence is taking a long time.  This slows Vrn1 expression under slow emergence and strange responses to delayed sowing
                EmergDurationFactor = np.exp(-0.015 * (DF.loc[d,'AccumTt']-90))
            DF.loc[d,'dHS'] = DF.loc[d,'Tt']/Params.BasePhyllo  * EmergDurationFactor
            DF.loc[d,'fPP'] = 0
            DF.loc[d,'Vrn2'] = 0
            
        else: # Crop emerged
            # Calculate delta haun stage
            DF.loc[d,'dHS'] = CalcdHS(DF.loc[d,'Tt'],DF.loc[d,'Pp'],DF.loc[d,'HS'],Params.BasePhyllo,Params.PhylloPpSens)
            
        #increment HS
        DF.loc[d,'HS'] += DF.loc[d,'dHS']
        
        # Calculate Vrn gene expression
        # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        if (IsReproductive==False):
            if (IsVernalised==False): #set as rates and factors for vegetative phase
                DF.loc[d,'dVrnB'] = Params.rVrnBVeg * DF.loc[d,'dHS']
                DF.loc[d,'dMaxApDev'] = Params.rApDevMVeg * DF.loc[d,'dHS']
                DF.loc[d,'PpVrn3Fact'] = Params.rVrn3Veg
            else:
                 #set as rates and factors for early reproductive phase
                DF.loc[d,'dVrnB'] = Params.rVrnBER * DF.loc[d,'dHS']
                DF.loc[d,'dMaxApDev'] = Params.rApDevMER * DF.loc[d,'dHS']
                DF.loc[d,'PpVrn3Fact'] = Params.rVrn3ER
            #Increment BaseVern and MaxVern
            DF.loc[d,'VrnB'] += DF.loc[d,'dVrnB']
            DF.loc[d,'MaxApDev'] = min(2,DF.loc[d,'MaxApDev'] + DF.loc[d,'dMaxApDev'])

            # Calculate daily cold response
            if (IsVernalised==False):
                DF.loc[d,'RelCold'] = CalcColdUpRegVrn1(DF.loc[d,'Tt'],1,1,Consts['k'])
                DF.loc[d,'dCold'] = DF.loc[d,'RelCold'] * DF.loc[d,'dVrnB'] * Params.rVrn1
                DF.loc[d,'Cold'] = max(0.0,DF.loc[d,'Cold'] + DF.loc[d,'dCold'])
                if DF.loc[d,'Cold'] > Params.MethalationThreshold:
                    DF.loc[d,'dVrn1'] = max(0,(DF.loc[d,'Cold'] - Params.MethalationThreshold)-(ColdYesterday - Params.MethalationThreshold))
            # Increment Vrn 1
            DF.loc[d,'Vrn1'] += DF.loc[d,'dVrn1']

            # Calcualte expression of photoperiod sensitive genes
            if (IsEmerged==True):  # Photoperiod sensitive genes only express after emergence
                DF.loc[d,'fPP'] = CalcPpResponse(DF.loc[d,'Pp'],0,1) #relative Pp, scaled between 0 at lower threshold and 1 at upper threshold
                DF.loc[d,'AccumSPp'] += DF.loc[d,'dVrnB'] * (1-DF.loc[d,'fPP'])  #accumulate short day exposure for reducing rVrn2
                DF.loc[d,'MaxVrn2'] = max(0,(Params.rVrn2 * DF.loc[d,'fPP']) - DF.loc[d,'AccumSPp'])
                DF.loc[d,'Vrn2'] = max(0,DF.loc[d,'MaxVrn2'] - (DF.loc[d,'VrnB'] + DF.loc[d,'Vrn1'])) 
                if (DF.loc[d,'Vrn2'] == 0): # express vrn3 relative to Pp if effective Vrn2 is down regulated to zero
                    DF.loc[d,'dVrn3'] = (DF.loc[d,'PpVrn3Fact']-1) * DF.loc[d,'fPP'] * DF.loc[d,'dVrnB']
                #Increment Vrn3
            DF.loc[d,'Vrn3'] += DF.loc[d,'dVrn3']

            # Increment todays Vrn expression values using deltas just calculated
            if (IsVernalised==False): #If not vernalised need to include Vrn2 and constrain to maxrate
                DF.loc[d,'ApDev'] = min(DF.loc[d,'MaxApDev'],max(0,DF.loc[d,'VrnB'] + DF.loc[d,'Vrn1'] + DF.loc[d,'Vrn3'] - DF.loc[d,'MaxVrn2'])) 
            else:
                DF.loc[d,'ApDev'] = DF.loc[d,'ApDev'] + DF.loc[d,'dVrnB'] + DF.loc[d,'dVrn3']

        # Set Haun stage variables
        # ^^^^^^^^^^^^^^^^^^^^^^^^
        if IsVernalised == False:
            DF.loc[d,'VIHS'] = DF.loc[d,'HS']
        if IsReproductive == False:
            DF.loc[d,'TSHS'] = DF.loc[d,'HS']
            DF.loc[d,'FLN'] = Consts['IntFLNvsTS'] + Consts['SlopeFLNvsTS'] * DF.loc[d,'TSHS']

        # Finally determine phenological stage
        # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        # Vernalisation saturation occurs when Vrn1 > the vernalisation threshold and Vrn2 expression is zero
        if (IsEmerged==True) and (DF.loc[d,'ApDev'] >= Consts['VIThreshold']) and (IsVernalised==False):
            IsVernalised  = True
            DF.loc[d,'Stage'] = 'Vern Sat'
            #DF.loc[d,'Vrn1'] = 0
        
        if (DF.loc[d,'ApDev'] >=  Consts['TSThreshold']) and (IsReproductive == False):
            IsReproductive = True;
            DF.loc[d,'Stage'] = 'Term Spike'
        
        #Work out if Flag leaf has appeared.
        if DF.loc[d,'HS'] >= DF.loc[d,'FLN']:
            IsAtFlagLeaf = True    
            DF.loc[d,'Stage'] = 'FlagLeaf'
    
        #Add states to dataframe
        IsDayOfEmergence = False
        
        DF.loc[d,'IsGerminated'] = IsGerminated
        DF.loc[d,'IsEmerged'] = IsEmerged
        DF.loc[d,'IsVernalised'] = IsVernalised
        DF.loc[d,'IsReproductive'] = IsReproductive
        DF.loc[d,'IsAtFlagLeaf'] = IsAtFlagLeaf
    
        # Increment day
        d += 1
        
    if (Out == 'FLN') and (IsAtFlagLeaf == True):
        return DF.iloc[-1,:]['FLN']
    else:
        return DF
    
def deriveVrnParams(c):
    data = pd.Series(dtype=np.float64)
    # Calculate the accumulated base phyllochrons at terminal spikelet for each treatment
    for pv in ['WS','CL','CS','WL']:
        data['TS_'+pv] = calcTSHS(c['FLN_'+pv],CampConstants['IntFLNvsTS'],CampConstants['SlopeFLNvsTS'])
    # Base Phyllochron
    data['BasePhyllo'] = c['BasePhyllo']
    # Phyllochron photoperiod sensitivity.  #This is not used in calculations but added so can be checked in outputs
    data['PhylloPpSens'] = c['PhylloPpSens']
    # Base Phyllochron duration of the Emergence Phase
    data['EmergDurat'] = c['TtEmerge']/data['BasePhyllo']
    # Base Phyllochron duration of vernalisation treatment
    data['VernTreatDurat'] = (c['VrnTreatDuration']*c['VrnTreatTemp'])/data['BasePhyllo']
    # Accumulated Base Phyllochrons when vernalisation treatment ended 
    data['EndVernTreat'] =  -data['EmergDurat'] + data['VernTreatDurat']
    # The minimum accumulated haunstage at which Vern saturation can occur
    data['MinVI'] = 1.1
    # The minimum haun stage duration from vern saturation to terminal spikelet
    data['MinVI->TS'] = min(3,data['TS_CL']-data['MinVI'])  #From Lincoln controlled environment data
    
    # Calculate the accumulated base phyllochrons at vernalisation saturation for each treatment
    data['VI_CL'] = data['TS_CL'] - data['MinVI->TS']   
    data['VI_WL'] = max(data['TS_WL'] - data['MinVI->TS'],data['VI_CL']) #Constrained so not earlier than the CL treatment
    data['VI_CS'] = min(data['VI_CL'],data['TS_CS']-data['MinVI->TS']) # Assume happens at the same time as VI_CL but not sooner than TS and MinVI->TS would allow 
    data['VI->TS_CS'] = data['TS_CS'] - data['VI_CS']
    data['VI_WS'] = max(data['TS_WS'] - data['VI->TS_CS'],data['VI_CL']) #Constrained so not earlier than the CL treatment
    
    # Calculate base and maximum rates and Pp sensitivities
    # Base Vrn delta during vegetative phase.  Assuming base Vrn expression starts at sowing and reaches 1 at VI for the WS treatment where no cold or Vrn3 (short Pp) to upregulate 
    data['rVrnBVeg'] = 1/(data['VI_WS']+data['EmergDurat'])
    # The fastest rate that Vrn can accumulate to reach saturation (VI).
    data['rApDevMVeg'] = 1/(data['VI_CS']+data['EmergDurat'])
    # Base Vrn delta during early reproductive phase.  Assuming Vrn expression increases by 1 between VI and TS and proceeds at a base rate where not Vrn3 upregulation (short PP)
    data['rVrnBER'] = 1/(data['TS_WS']-data['VI_WS'])
    # The maximum Vrn delta during the early reproductive phase.  Assuming Vrn increases by 1 bewtween VI and TS and proceeds at the maximum rate under long photoperiod
    data['rApDevMER'] = 1/data['MinVI->TS']
    # The relative increase in delta Vrn cuased by Vrn3 expression under long photoperiod during early reproductive phase
    data['rVrn3ER'] = data['rApDevMER']/data['rVrnBER']
    # The relative increase in delta Vrn1 caused by Vrn3 expression under long Pp during vegetative phase.  Same as rVrn3ER unless VI_WL is small
    data['rVrn3Veg'] = (1/data['VI_WL'])/data['rVrnBVeg'] 
    data['rVrn3Veg'] = max(data['rVrn3ER'],data['rVrn3Veg']) 
    
    # Calculate Maximum Vrn2 expression under long photoperiods
    # The rate of Vrn expression before VI under long Pp when Vrn2 = 0 is the product of Vrn3 and base Vrn
    # Therefore the haun stage duration when effective baseVrn x Vrn3 is up regulating during the vegetative phase under long photoperiod without vernalisation is a funciton of the VI target (1) and the rate
    data['URVrn3HSWL'] = 1/(data['rVrnBVeg'] * data['rVrn3Veg'])
    # The accumulated Haun stages when Vrn2 expression is ended and effective baseVrn x Vrn3 expression starts under long Pp without vernalisation is vrn1xVrn3Durat prior to VI
    data['DRVrn2HSWL'] = max(0,data['VI_WL'] - data['URVrn3HSWL']) 
    # The Vrn2 Expression that must be matched by Vrn1 before effective Vrn3 expression starts.  
    data['rVrn2'] = (data['DRVrn2HSWL']+data['EmergDurat']) * data['rVrnBVeg']
    
    # Calculate cold upregulation of Vrn1
    # The amount of protaganistic vrn expression required to reach VI
    data['VrnPVI_CL'] = CampConstants['VIThreshold'] + data['rVrn2']
    # The amount of Vrn expression at base rate in the CL treatment between sowing and when Vrn2 is suppressed 
    data['VrnBVeg_CL'] = (data['EmergDurat']+data['VI_CL']) * data['rVrnBVeg']
    # The amount of persistant (methalated) Vrn1 upregulated due to cold at the end of the vernalisation treatment for CL
    # Is VrnPVI_CL less base vrn expression up to VI.  Assuming Vrn3 expression prior to VI is neglegable in this case 
    data['Vrn1_CL'] = data['VrnPVI_CL'] - data['VrnBVeg_CL'] 
    # Cold threshold required before Vrn1 starts methalating.  Based on Brooking and Jamieson data the lag duration is the same length as the duration of response up to full vernalisation
    # Therefore we assume the methalation threshold must be the same size as the amount of vrn1 that is required to give full persistend vernalisation response.
    data['MethalationThreshold'] = data['Vrn1_CL']
    # Haun stage duration of cool treatment, may be less than treatment duration if treatment goes past VI
    data['CTHS'] =  min(data['VernTreatDurat'],data['VI_CL']+data['EmergDurat'])
    # The rate of Vrn1 expression under cold treatment calculated from the amopunt of cold vrn1 up regulation apparante plus the methalation threshold
    data['DVrn1_CL'] = (data['Vrn1_CL']+data['MethalationThreshold'])/data['CTHS'] 
    # The upregulation of Vrn1 expression above base rate at 0oC
    data['DVrn1Max'] = data['DVrn1_CL']/ np.exp(c['k'] * c['VrnTreatTemp']) 
    # The relative increase in delta Vrn1 caused by cold upregulation of Vrn1
    data['rVrn1'] = data['DVrn1Max'] /data['rVrnBVeg']
                                 
    return data

def plotFLNs(CL,CS,WS,WL,Axis,fs,maxFLN=20):
    width = 0.4
    ind = np.arange(4) + width
    plt.bar(ind+.4, [CL,CS,WS,WL], width,
            edgecolor=['b','b','r','r'], color = ['b','w','w','r'],
            linewidth=3,alpha=0.3)
    plt.ylabel('Final Leaf Number',fontsize=12)
    plt.tick_params(labelsize=fs)
    Axis.set_xticks(ind+width)
    Axis.set_xticklabels(['$CL$','$CS$', '$WS$', '$WL$'])
    plt.ylim(0,maxFLN)
    plt.xlim(0.5,4.1)
    Axis.spines['right'].set_visible(False)
    Axis.spines['top'].set_visible(False)
    plt.tick_params(axis='x', which='both', bottom=True,top=False, labelbottom=True)
    plt.tick_params(axis='y', which='both', left=True,right=False, labelleft=True)

def addArrow(Xs,Ys,col='k'):
    bx = Xs[0]
    by = Ys[0]
    dx = Xs[1] 
    dy = Ys[1]
    prop = dict(arrowstyle="-|>,head_width=0.2,head_length=0.5",color=col,lw=1,ls=':',
            shrinkA=0,shrinkB=0)
    plt.annotate("",xy=(dx,dy),xytext=(bx,by), arrowprops=prop)
    
def plotLNlines(CL,CS,WS,WL,fs):
    MinLN = CL
    PpLN = max(0,CS - CL)
    CvLN = max(0,WS - MinLN - PpLN)
    PvLN = WL - MinLN - CvLN
    
    #MinLN
    addArrow([.6,3],[MinLN]*2)
    plt.plot([0.6]*2,[0,MinLN],'-',color='k',lw=4)
    plt.text(0.7,CL-2,'$MinLN=$'+"{:.1f}".format(MinLN),fontsize=fs,verticalalignment='center')
    #PpLN
    if PpLN>0.5:
        addArrow([1.6,2.6],[MinLN+PpLN]*2,'darkorange')
        plt.plot([1.6,1.6],[CS,CL],'-',color='darkorange',lw=4)
        plt.text(1.5,max((CL+CS)/2,CL+.75),'$PpLN=$'+"{:.1f}".format(PpLN),
                 fontsize=fs,verticalalignment='center',horizontalalignment='right',color='darkorange')
    else:
        plt.text(1.3,(CL+CS)/2,'$PpLN=0$',fontsize=fs,verticalalignment='bottom',color='darkorange')
    #CvLN
    if CvLN > 0.5:
        plt.plot([2.6,2.6],[CS,MinLN+PpLN+CvLN],'-',color='darkblue',lw=4)
        plt.text(2.5,max((CS+WS)/2,CS+.75),'$CvLN=$'+"{:.1f}".format(CvLN),
                 fontsize=fs, verticalalignment='center',horizontalalignment='right',color='darkblue')
        
        plt.plot([3,3],[MinLN,MinLN+CvLN],'-',color='darkblue',lw=4)
        addArrow([2.6,3],[MinLN+PpLN+CvLN*.5,MinLN+CvLN*.5],'darkblue')
        addArrow([3,3.2],[MinLN + CvLN]*2,'darkblue')
    else:
        plt.text(2.1,(CS+WS)/2,'$CvLN=0$',fontsize=fs, verticalalignment='bottom',color='darkblue')
    #PvLN
    if ((PvLN > 0.5)or(PvLN<-0.5)):
        plt.plot([3.2,3.2],[CL+CvLN,WL],'-',color='forestgreen',lw=4)
        plt.text(3.3,max(CL+CvLN + PvLN/2,WL+0.75),'$PvLN=$'+"{:.1f}".format(PvLN),
                 fontsize=fs, verticalalignment='center',color = 'darkgreen')
    else:
        plt.text(3.3,WL+0.75,'$PvLN=0$',fontsize=fs, verticalalignment='center',color = 'darkgreen')
    addArrow([3.6,3.2],[WL,WL],'darkgreen')
    
def ExpressionPlot(data,c):
    lowestTS = 16
    highestTS = 0
    pos = 0
    #Plot TS for all treats at apparent Vrn1 expression of 2.0
    for pv in ['WS','CL','CS','WL']:
        #plt.plot(CampInputs.loc[c,'FLN_'+pv],2.5,'*',mfc = fills[pos],mec=colors[pos],ms=10)
        plt.plot([data['TS_'+pv]],[2],'o',mfc = fills[pos],mec=colors[pos],ms=10)
        lowestTS = min(lowestTS,data['TS_'+pv])
        highestTS = max(highestTS,data['TS_'+pv])
        plt.plot([data['TS_'+pv]],[2.1],'v',color = 'k',ms=7)
        plt.plot([data['TS_'+pv]]*2,[2.1,2.2],'-',color = 'k',ms=7)
        pos+=1
    plt.plot([lowestTS,highestTS * 1.2],[2.2,2.2],'-',color = 'k',ms=7)
    plt.text(highestTS * 1.25, 2.2,'TS',horizontalalignment='center')
    
    #Plot VI_CL treatments at apparent Vrn1 expression of 1.0
    plt.plot([data['VI_CL']],[1],'o',mec='b',mfc='b',ms=10)
    plt.plot([data['VI_CL'],data['TS_CL']],[2,2],'--',color='k')
    plt.text((data['VI_CL']+data['TS_CL'])/2,2.05,'MinVI->TS',horizontalalignment='center')
    plt.plot([data['VI_CL']]*2,[1,2],'--',color='k')
    plt.text(data['VI_CL'],1.05,'VI_CL',horizontalalignment='center',verticalalignment='bottom')
    #Plot VI_LN treatments at apparent Vrn1 expression of 1.0
    plt.plot([data['VI_WL'],data['TS_WL']],[2,2],'--',color='k')
    plt.text((data['VI_WL']+data['TS_WL'])/2,2.05,'MinVI->TS',horizontalalignment='center')
    plt.plot([data['VI_WL']]*2,[1,2],'--',color='k')
    plt.plot([data['VI_WL']],[1],'o',mec='r',mfc='r',ms=10)
    plt.text(data['VI_WL'],1.05,'VI_WL',horizontalalignment='center',verticalalignment='bottom')
    #Plot VI_CS treatments at apparent Vrn1 expression of 1.0
    plt.plot([data['VI_CS']],[1],'o',mec='b',mfc='w',ms=10)
    plt.text(data['VI_CS'],0.95,'VI_CS',horizontalalignment='center',verticalalignment='top')
    plt.plot([data['VI_CS'],data['TS_CS']],[1.8,1.8],':',color='k')
    plt.plot([data['VI_CS'],data['VI_CS'],np.nan,data['TS_CS'],data['TS_CS']],[1,1.8,np.nan,1.8,2],':',color='k')
    plt.text((data['VI_CS']+data['TS_CS'])/2,1.85,'VI->Ts_S',horizontalalignment='center')
    #Plot VI_WS treatments at apparent Vrn1 expression of 1.0
    plt.plot([data['VI_WS']],[1],'o',mec='r',mfc='w',ms=10)
    plt.text(data['VI_WS'],0.95,'VI_WS',horizontalalignment='center',verticalalignment='top')
    plt.plot([data['VI_WS'],data['TS_WS']],[1.65,1.65],':',color='k')
    plt.plot([data['VI_WS'],data['VI_WS'],np.nan,data['TS_WS'],data['TS_WS']],[1,1.65,np.nan,1.65,2],':',color='k')
    plt.text((data['VI_WS']+data['TS_WS'])/2,1.7,'VI->Ts_S',horizontalalignment='center')
    
    #Plot base Vern1 Rate Vegetative
    plt.plot([-data['EmergDurat'],data['VI_WS']],[0,(data['VI_WS']+data['EmergDurat'])*data['rVrnBVeg']],'-',color='g')
    MidP = (-data['EmergDurat']+data['VI_WS'])/2
    plt.plot([MidP,MidP+1],[0.5,0.5],'--',color='g')
    plt.text(MidP+.5,0.53,'rVrnBVeg',color='g')
    
    #Plot max Vrn1 rate vegetative
    plt.plot([-data['EmergDurat'],data['VI_CS']],[0,(data['VI_CS']+data['EmergDurat'])*data['rApDevMVeg']],'-',color='gold')
    MidP = (data['VI_CL']+data['TS_CL'])/2
    plt.plot([MidP,MidP+1],[1.5,1.5],'-',color='gold')
    plt.text(MidP+.5,.53,'maxDVrnVeg',color='gold')
    
    #Plot base Vern1 Rate Early reproductive
    plt.plot([data['VI_WS'],data['TS_WS']],[1,1+(data['TS_WS'] - data['VI_WS'])*data['rVrnBER']],'-',color='g')
    MidP = (data['VI_WS']+data['TS_WS'])/2
    plt.plot([MidP,MidP+1],[1.5,1.5],'--',color='g')
    plt.text(MidP+.5,1.53,'rVrnBER',color='g')
    
    #Plot max Vrn1 ER
    plt.plot([data['VI_CL'],data['TS_CL']],[1,1+(data['TS_CL'] - data['VI_CL'])*data['rApDevMER']],'-',color='gold')
    MidP = (data['VI_CL']+data['TS_CL'])/2
    plt.plot([MidP,MidP+1],[1.5,1.5],'-',color='gold')
    plt.text(MidP+.5,1.53,'rApDevMER',color='gold')
    
    # Plot end of vernalisation timing
    data['endVern'] = min(data['EndVernTreat'],data['VI_CL'])
    plt.plot([data['endVern']]*2,[-1,2],'--',color='brown')
    plt.text(data['endVern'],-0.1,'endVern',horizontalalignment='center',color='brown')
    
    #Extrapolate vrn1 back from VI_WL to show when it started effective expression
    plt.plot([data['DRVrn2HSWL'],data['VI_WL']],[0,data['URVrn3HSWL'] * (data['rVrnBVeg'] * data['rVrn3Veg'])],'--',color='grey')
    MidP = data['DRVrn2HSWL']+data['URVrn3HSWL']*0.1
    plt.plot([MidP,MidP+1],[0.1]*2,'--',color='grey')
    plt.text(MidP+.2,0.15,'dvrnxVrn3',color='grey')
    
    #Extrapolate BaseVrn1 back to MinVI to show how much Vrn1 was required to overcome Vrn2
    plt.plot([0,data['DRVrn2HSWL']],[data['rVrn2'],0],'--',color='Orange')
    plt.text(0,min(2.15,data['rVrn2']),'Vrn2',color='orange')
    
    #The amount of Vrn1 required to reach VI is light is 1 + rVrn2
    plt.plot([0,data['VI_CL']],[1+data['rVrn2']]*2,'--',color='k')
    plt.text(data['VI_CL']-.2,1+data['rVrn2']+.03,'Vrn Target',horizontalalignment='right',verticalalignment='bottom')
    #the amount required from cold up regulation is the above less base Vrn1 until the end of Vrn2 less base * Vrn3 between end vrn2 and VI
    plt.plot([0,data['VI_CL']],[1+data['rVrn2']-data['VrnBVeg_CL']]*2,'-',color='g')
    plt.text(data['VI_CL']-.2,data['Vrn1_CL']+.03,'less baseVrn',horizontalalignment='right',verticalalignment='bottom')
    
    #RelMethTime = camp.CampConstants['MethalationThreshold']/(data['Vrn1_CL']+camp.CampConstants['MethalationThreshold'])
    StartMethHS = -data['EmergDurat'] + data['CTHS'] * 0.5 
    plt.plot([StartMethHS,data['endVern']],[0,data['Vrn1_CL']],'-',color='cyan')
    plt.text(-data['EmergDurat'],data['Vrn1_CL'],'Vrn1_CL',horizontalalignment='right',rotation=90,verticalalignment='top',color='c')
    
    plt.ylabel('Apparent Vrn Expression')
    plt.xlabel('Haun Stage')
    plt.ylim(-.2,2.5)
    plt.xlim(-2,20)
    plt.text(19,0.05,c,horizontalalignment='right')
    
def plotVITS(data, level, c):
    lowestTS = 16
    highestTS = 0
    pos = 0
    #Plot TS for all treats at apparent Vrn1 expression of 2.0
    for pv in ['WS','CL','CS','WL']:
        #plt.plot(CampInputs.loc[c,'FLN_'+pv],2.5,'*',mfc = fills[pos],mec=colors[pos],ms=10)
        plt.plot([data['TS_'+pv]],[2],'o',mfc = fills[pos],mec=colors[pos],ms=10)
        plt.text(data['TS_'+pv],2.25,pv,horizontalalignment='center')
        lowestTS = min(lowestTS,data['TS_'+pv])
        highestTS = max(highestTS,data['TS_'+pv])
        plt.plot([data['TS_'+pv]],[2.1],'v',color = 'k',ms=7)
        plt.plot([data['TS_'+pv]]*2,[2.1,2.2],'-',color = 'k',ms=7)
        pos+=1
    plt.text(-1, 2.05,r'$TS$',horizontalalignment='center',verticalalignment='center')
    plt.plot([-2,20],[2,2],ls='dotted',color='k')
    plt.ylabel('Apparent Vrn Expression')
    plt.xlabel('Haun Stage')
    plt.ylim(0,2.5)
    plt.xlim(-2,18)
    plt.text(17,0.05,c,horizontalalignment='right')
    if(level>1):
        plt.text(-1, 1.05,r'$VI$',horizontalalignment='center',verticalalignment='center')
        plt.plot([-2,20],[1,1],ls='dotted',color='k')
        #Plot VI_CL treatments at apparent Vrn1 expression of 1.0
        plt.plot([data['VI_CL']],[1],'o',mec='b',mfc='b',ms=10)
        plt.plot([data['VI_CL']+.1,data['TS_CL']-.1],[1.75,1.75],'>--',color='k')
        plt.text((data['VI_CL']+data['TS_CL'])/2,1.75,r'$MinER^{HS}$',horizontalalignment='center',verticalalignment='bottom')
        plt.plot([data['VI_CL'],data['VI_CL'],np.nan,data['TS_CL'],data['TS_CL']],[1,1.75,np.nan,1.75,2],ls='dotted',color='k')
        plt.text(data['VI_CL'],.67,'CL',horizontalalignment='center',verticalalignment='bottom')
        plt.plot([data['VI_CL']],[0.9],'^',color = 'k',ms=7)
        plt.plot([data['VI_CL']]*2,[0.9,0.8],'-',color = 'k',ms=7)
       #Plot VI_LN treatments at apparent Vrn1 expression of 1.0
        plt.plot([data['VI_WL']+.1,data['TS_WL']-.1],[1.55,1.55],'>--',color='k')
        plt.text((data['VI_WL']+data['TS_WL'])/2,1.55,r'$MinER^{HS}$',horizontalalignment='center',verticalalignment='bottom')
        plt.plot([data['VI_WL'],data['VI_WL'],np.nan,data['TS_WL'],data['TS_WL']],[1,1.55,np.nan,1.55,2],ls='dotted',color='k')
        plt.plot([data['VI_WL']],[1],'o',mec='r',mfc='r',ms=10)
        plt.text(data['VI_WL'],.67,'WL',horizontalalignment='center',verticalalignment='bottom')
        plt.plot([data['VI_WL']],[0.9],'^',color = 'k',ms=7)
        plt.plot([data['VI_WL']]*2,[0.9,0.8],'-',color = 'k',ms=7)
        if(level>2):    
            #Plot VI_CS treatments at apparent Vrn1 expression of 1.0
            plt.plot([data['VI_CS']],[1],'o',mec='b',mfc='w',ms=10)
            plt.text(data['VI_CS'],0.67,'CS',horizontalalignment='center',verticalalignment='top')
            plt.plot([data['VI_CS']+.1,data['TS_CS']-.1],[1.35,1.35],'>-.',color='k')
            plt.plot([data['VI_CS'],data['VI_CS'],np.nan,data['TS_CS'],data['TS_CS']],[1,1.35,np.nan,1.35,2],':',color='k')
            plt.text((data['VI_CS']+data['TS_CS'])/2,1.35,r'$MaxER^{HS}$',horizontalalignment='center',verticalalignment='bottom')
            #Plot VI_WS treatments at apparent Vrn1 expression of 1.0
            plt.plot([data['VI_WS']],[1],'o',mec='r',mfc='w',ms=10)
            plt.text(data['VI_WS'],0.67,'WS',horizontalalignment='center',verticalalignment='bottom')
            plt.plot([data['VI_WS']+.1,data['TS_WS']-.1],[1.15,1.15],'>-.',color='k')
            plt.plot([data['VI_WS'],data['VI_WS'],np.nan,data['TS_WS'],data['TS_WS']],[1,1.15,np.nan,1.15,2],':',color='k')
            plt.text((data['VI_WS']+data['TS_WS'])/2,1.15,r'$MaxER^{HS}$',horizontalalignment='center',verticalalignment='bottom')
            plt.plot([data['VI_WS']],[0.9],'^',color = 'k',ms=7)
            plt.plot([data['VI_WS']]*2,[0.9,0.8],'-',color = 'k',ms=7)
            if(level>3):
                #Plot base Vern1 Rate Vegetative
                plt.plot([-data['EmergDurat'],data['VI_WS']],[0,(data['VI_WS']+data['EmergDurat'])*data['rVrnBVeg']],'-',color='g')
                MidP = (-data['EmergDurat']+data['VI_WS'])/2
                plt.plot([MidP,MidP+1],[0.5,0.5],'--',color='g')
                plt.text(MidP+.5,0.53,'rVrnBVeg',color='g')
                #Plot max Vrn1 rate vegetative
                plt.plot([-data['EmergDurat'],data['VI_CS']],[0,(data['VI_CS']+data['EmergDurat'])*data['MaxDVrnVeg']],'-',color='gold')
                MidP = (data['VI_CL']+data['TS_CL'])/2
                plt.plot([MidP,MidP+1],[1.5,1.5],'-',color='gold')
                plt.text(MidP+.5,1.53,'maxDVrnER',color='gold')
                #Plot base Vern1 Rate Early reproductive
                plt.plot([data['VI_WS'],data['TS_WS']],[1,1+(data['TS_WS'] - data['VI_WS'])*data['rVrnBER']],'-',color='g')
                MidP = (data['VI_WS']+data['TS_WS'])/2
                plt.plot([MidP,MidP+1],[1.5,1.5],'--',color='g')
                plt.text(MidP+.5,1.53,'rVrnBER',color='g')

                #Plot max Vrn1 ER
                plt.plot([data['VI_CL'],data['TS_CL']],[1,1+(data['TS_CL'] - data['VI_CL'])*data['MaxDVrnER']],'-',color='gold')
                MidP = (data['VI_CL']+data['TS_CL'])/2
                plt.plot([MidP,MidP+1],[1.5,1.5],'-',color='gold')
                plt.text(MidP+.5,1.53,'MaxDVrnER',color='gold')
                if(level>4):
                    # Plot end of vernalisation timing
                    data['endVern'] = min(data['EndVernTreat'],data['VI_CL'])
                    plt.plot([data['endVern']]*2,[-1,2],'--',color='brown')
                    plt.text(data['endVern'],-0.1,'endVern',horizontalalignment='center',color='brown')

                    #Extrapolate vrn1 back from VI_WL to show when it started effective expression
                    plt.plot([data['endVrn2_WL'],data['VI_WL']],[0,data['vrnxVrn3Durat'] * data['dvrnxVrn3']],'--',color='grey')
                    MidP = data['endVrn2_WL']+data['vrnxVrn3Durat']*0.1
                    plt.plot([MidP,MidP+1],[0.1]*2,'--',color='grey')
                    plt.text(MidP+.2,0.15,'dvrnxVrn3',color='grey')

                    #Extrapolate BaseVrn1 back to MinVI to show how much Vrn1 was required to overcome Vrn2
                    plt.plot([0,data['endVrn2_WL']],[data['rVrn2'],0],'--',color='Orange')
                    plt.text(0,min(2.15,data['rVrn2']),'Vrn2',color='orange')

                    #The amount of Vrn1 required to reach VI is light is 1 + rVrn2
                    plt.plot([0,data['VI_CL']],[1+data['rVrn2']]*2,'--',color='k')
                    plt.text(data['VI_CL']-.2,1+data['rVrn2']+.03,'Vrn Target',horizontalalignment='right',verticalalignment='bottom')
                    #the amount required from cold up regulation is the above less base Vrn1 until the end of Vrn2 less base * Vrn3 between end vrn2 and VI
                    plt.plot([0,data['VI_CL']],[1+data['rVrn2']-data['VrnBVeg_CL']]*2,'-',color='g')
                    plt.text(data['VI_CL']-.2,data['Vrn1_CL']+.03,'less baseVrn',horizontalalignment='right',verticalalignment='bottom')

                    #RelMethTime = camp.CampConstants['MethalationThreshold']/(data['Vrn1_CL']+camp.CampConstants['MethalationThreshold'])
                    StartMethHS = -data['EmergDurat'] + data['Vrn1HS_CL'] * 0.5 
                    plt.plot([StartMethHS,data['endVern']],[0,data['Vrn1_CL']],'-',color='cyan')
                    plt.text(-data['EmergDurat'],data['Vrn1_CL'],'Vrn1_CL',horizontalalignment='right',rotation=90,verticalalignment='top',color='c')
    
def boundPlots(data, ax, c):
    lowestTS = 16
    highestTS = 0
    pos = 0
    
    #Plot TS for all treats at apparent Vrn1 expression of 2.0
    for pv in ['WS','CL','CS','WL']:
        #plt.plot(CampInputs.loc[c,'FLN_'+pv],2.5,'*',mfc = fills[pos],mec=colors[pos],ms=10)
        plt.plot([data['TS_'+pv]],[2],'o',mfc = fills[pos],mec=colors[pos],ms=10)
        plt.text(data['TS_'+pv],2.25,pv,horizontalalignment='center')
        lowestTS = min(lowestTS,data['TS_'+pv])
        highestTS = max(highestTS,data['TS_'+pv])
        plt.plot([data['TS_'+pv]],[2.1],'v',color = 'k',ms=7)
        plt.plot([data['TS_'+pv]]*2,[2.1,2.2],'-',color = 'k',ms=7)
        pos+=1
    plt.text(-1, 2.05,r'$TS$',horizontalalignment='center',verticalalignment='center')
    plt.plot([-2,20],[2,2],ls='dotted',color='k')
    plt.ylabel('Apparent Vrn Expression')
    plt.xlabel('Haun Stage')
    plt.ylim(0,2.5)
    plt.xlim(-2,18)
    plt.text(17,0.05,c,horizontalalignment='right')
    
    plt.text(-1, 1.05,r'$VI$',horizontalalignment='center',verticalalignment='center')
    plt.plot([-2,20],[1,1],ls='dotted',color='k')
    #Plot VI_CL treatments at apparent Vrn1 expression of 1.0
    plt.plot([data['VI_CL']],[1],'o',mec='b',mfc='b',ms=10)
    plt.text(data['VI_CL'],1.3,'CL',horizontalalignment='center',verticalalignment='bottom')
    plt.plot([data['VI_CL']],[1.1],'v',color = 'k',ms=7)
    plt.plot([data['VI_CL']]*2,[1.1,1.2],'-',color = 'k',ms=7)
   #Plot VI_WL treatments at apparent Vrn1 expression of 1.0
    plt.plot([data['VI_WL']],[1],'o',mec='r',mfc='r',ms=10)
    plt.text(data['VI_WL'],.67,'WL',horizontalalignment='center',verticalalignment='bottom')
    plt.plot([data['VI_WL']],[0.9],'^',color = 'k',ms=7)
    plt.plot([data['VI_WL']]*2,[0.9,0.8],'-',color = 'k',ms=7)
    
    #Plot VI_CS treatments at apparent Vrn1 expression of 1.0
    plt.plot([data['VI_CS']],[1],'o',mec='b',mfc='w',ms=10)
    plt.text(data['VI_CS'],1.3,'CS',horizontalalignment='center',verticalalignment='top')
    
    #Plot VI_WS treatments at apparent Vrn1 expression of 1.0
    plt.plot([data['VI_WS']],[1],'o',mec='r',mfc='w',ms=10)
    plt.text(data['VI_WS'],0.67,'WS',horizontalalignment='center',verticalalignment='bottom')
    # plt.plot([data['VI_WS']+.1,data['TS_WS']-.1],[1.15,1.15],'>-.',color='k')
    # plt.plot([data['VI_WS'],data['VI_WS'],np.nan,data['TS_WS'],data['TS_WS']],[1,1.15,np.nan,1.15,2],':',color='k')
    # plt.text((data['VI_WS']+data['TS_WS'])/2,1.15,r'$MaxER^{HS}$',horizontalalignment='center',verticalalignment='bottom')
    plt.plot([data['VI_WS']],[0.9],'^',color = 'k',ms=7)
    plt.plot([data['VI_WS']]*2,[0.9,0.8],'-',color = 'k',ms=7)
            
    basepos = 0.25
    #Plot base Vern1 Rate Vegetative
    plt.plot([-data['EmergDurat'],data['VI_WS']],[0,(data['VI_WS']+data['EmergDurat'])*data['rVrnBVeg']],'-',color='g')
    MidP = ((data['EmergDurat']+data['VI_WS'])* basepos)-data['EmergDurat']
    ang = np.arctan(1/(data['EmergDurat']+data['VI_WS']))*(180/np.pi)
    arc = patches.Arc([MidP,basepos],1.0,.125,angle=0.0,theta1=0.0,theta2=ang,linewidth=2, fill=False, color='g')
    ax.add_patch(arc)
    plt.plot([MidP,MidP+1],[basepos]*2,'-',color='g')
    plt.text(MidP+.7,basepos+.03,r'$rVrnB_{Veg}$',color='g')
    
    maxpos = 0.75
    #Plot max Vrn1 rate vegetative
    plt.plot([-data['EmergDurat'],data['VI_CS']],[0,(data['EmergDurat']+data['VI_CS'])*data['rApDevMVeg']],'-',color='gold')
    MidP = ((data['EmergDurat']+data['VI_CS']) * maxpos)-data['EmergDurat']
    ang = np.arctan(1/(data['EmergDurat']+data['VI_CS']))*(180/np.pi)
    arc = patches.Arc([MidP,maxpos],1.0,.125,angle=0.0,theta1=0.0,theta2=ang,linewidth=2, fill=False, color='gold')
    ax.add_patch(arc)
    plt.plot([MidP,MidP+1],[maxpos]*2,'-',color='gold')
    plt.text(MidP+.7,maxpos+0.03,r'$rApDevM_{Veg}$',color='gold', horizontalalignment = 'left')
    
    #Plot base Vern1 Rate Early reproductive
    plt.plot([data['VI_WS'],data['TS_WS']],[1,1+(data['TS_WS'] - data['VI_WS'])*data['rVrnBER']],'-',color='g')
    MidP =  data['VI_WS'] + (data['VI->TS_CS'] * basepos)
    ang = np.arctan(1/data['VI->TS_CS'])*(180/np.pi)
    arc = patches.Arc([MidP,1+basepos],1.0,.125,angle=0.0,theta1=0.0,theta2=ang,linewidth=2, fill=False, color='g')
    ax.add_patch(arc)
    plt.plot([MidP,MidP+1],[1+basepos]*2,'-',color='g')
    plt.text(MidP+.7,1+basepos+.03,r'$rVrnB_{ER}$',color='g')

    # #Plot max Vrn1 ER
    plt.plot([data['VI_CL'],data['TS_CL']],[1,1+(data['TS_CL'] - data['VI_CL'])*data['rApDevMER']],'-',color='gold')
    MidP = data['VI_CL']+(data['MinVI->TS']* maxpos)
    ang = np.arctan(1/(data['EmergDurat']+data['VI_CS']))*(180/np.pi)
    arc = patches.Arc([MidP,1+maxpos],1.0,.125,angle=0.0,theta1=0.0,theta2=ang,linewidth=2, fill=False, color='gold')
    ax.add_patch(arc)
    plt.plot([MidP,MidP+1],[1+maxpos]*2,'-',color='gold')
    plt.text(MidP+.7,1+maxpos+0.03,r'$rApDevM_{ER}$',color='gold',)
    
    plt.plot(-data['EmergDurat'],0.03,'v',color = 'k',ms=7)
    plt.plot([-data['EmergDurat']]*2,[0,.2],'-',color = 'k',ms=7)
    plt.text(-data['EmergDurat'],.25,'Imbibation', rotation = 90, verticalalignment='bottom',horizontalalignment = 'center')
    
    plt.plot(0,0.03,'v',mec = 'k',mfc='white',ms=7)
    plt.plot([0]*2,[.08,.4],'--',color = 'k',ms=7)
    plt.text(0,.45,'Emergence', rotation = 90, verticalalignment='bottom',horizontalalignment = 'center')
    
def Vrn1Plots(data,ax, c):
    lowestTS = 16
    highestTS = 0
    pos = 0

    plt.ylabel('Apparent Vrn Expression')
    plt.xlabel('Haun Stage')
    plt.ylim(0,1.8)
    plt.xlim(-1.2,5)
    plt.text(5,0.05,c,horizontalalignment='right')
    
    plt.text(4.8, 1.05,r'$VI$',horizontalalignment='center',verticalalignment='center')
    plt.plot([-2,20],[1,1],ls='dotted',color='k')

    plt.plot([data['VI_CS']],[1],'o',mec='b',mfc='b',ms=10)
    plt.text(data['VI_CS'],.67,'CS',horizontalalignment='center',verticalalignment='bottom')
    plt.plot([data['VI_CS']],[0.9],'^',color = 'k',ms=7)
    plt.plot([data['VI_CS']]*2,[0.9,0.8],'-',color = 'k',ms=7)
 
    #Extrapolate BaseVrn1 back to MinVI to show how much Vrn1 was required to overcome Vrn2
    plt.text(data['VI_CS']+.92,(2+data['rVrn2'])/2,r'$rVrn2$',color='crimson')
    plt.plot([data['VI_CS']+.9]*2,[1.02,0.98+data['rVrn2']], '^--', color='crimson')
    
    basepos = 0.12
    #Plot base Vern1 Rate Vegetative
    plt.plot([-data['EmergDurat'],data['VI_CS']],[0,(data['VI_CS']+data['EmergDurat'])*data['rVrnBVeg']],'-',color='g')
    MidP = ((data['EmergDurat']+data['VI_WS'])* basepos)-data['EmergDurat']
    ang = np.arctan(1/(data['EmergDurat']+data['VI_WS']))*(180/np.pi)
    arc = patches.Arc([MidP,basepos],1.0,.125,angle=0.0,theta1=0.0,theta2=ang,linewidth=2, fill=False, color='g')
    ax.add_patch(arc)
    plt.plot([MidP,MidP+1],[basepos]*2,'-',color='g')
    plt.text(MidP+.7,basepos+.03,r'$rVrnB_{Veg}$',color='g')
    
    plt.plot([data['VI_CS'],data['VI_CS']],[0.02,(data['VI_CS']+data['EmergDurat'])*data['rVrnBVeg']-.02],'^-.',color='g')
    plt.text(data['VI_CS']+.02,0.2,r'$\Sigma^0_{VI}VrnB_{Veg}$',horizontalalignment='left',verticalalignment='bottom',color='g')
    
    plt.plot([1.5,1.5],[1.02+data['rVrn2']-data['VrnBVeg_CL'],data['rVrn2']+.98],'^-.',color='g')
    plt.text(1.52,1.2,r'$\Sigma^0_{VI}VrnB_{Veg}$',horizontalalignment='left',verticalalignment='bottom',color='g')
    
    plt.plot([-2,data['VI_CS']+.9],[1+data['rVrn2']]*2,'--',color='k')
    plt.text(-1,1.02+data['rVrn2'],r'$Vrn^PVI_{CL}$')
    
    plt.plot([-2,1.5],[1+data['rVrn2']-data['VrnBVeg_CL']]*2,'-.',color='k')
    plt.text(-1,1.02+data['rVrn2']-data['VrnBVeg_CL'],'$Vrn1_{CL}$')
    
    plt.plot(0,0.03,'v',mec = 'b',mfc='b',ms=7)
    plt.plot([0,0,np.nan,0,0,np.nan,0,0],[.03,.1,np.nan,.45,.55,np.nan,.7,1.0+data['rVrn2']-data['VrnBVeg_CL']],'--',color = 'b',ms=7)
    plt.text(0,.15,'End Vern', rotation = 90, verticalalignment='bottom',horizontalalignment = 'center',color='b')
    
    MethHS = 1/(1+1.+data['rVrn2']-data['VrnBVeg_CL'])
    Vrn1Needed = 1.0+data['rVrn2']-data['VrnBVeg_CL']
    basepos = 0.6
    plt.plot([-data['EmergDurat']*MethHS,0],[0,Vrn1Needed],'-',color='b')
    MidP = (-data['EmergDurat']*MethHS)/2 
    ang = np.arctan((Vrn1Needed)/(data['EmergDurat']*MethHS))*(180/np.pi)
    arc = patches.Arc([MidP,basepos],0.5,.125,angle=0.0,theta1=0.0,theta2=ang,linewidth=2, fill=False, color='b')
    ax.add_patch(arc)
    plt.plot([MidP,MidP+.5],[basepos]*2,'-',color='b')
    plt.text(MidP+0.35,basepos+.05,r'$\Delta Vrn1_{CL}$',color='b')
    ang = np.arctan(((Vrn1Needed))/(data['EmergDurat']*MethHS))*(180/np.pi)*1.23
    plt.text(-data['EmergDurat']*MethHS+.04,.15,'Methalated Vrn1', rotation = ang, verticalalignment='bottom',horizontalalignment = 'center',color='b')
        
    plt.plot([-data['EmergDurat'],-data['EmergDurat']*MethHS],[0,1.0],'-',color='deepskyblue')
    plt.text(-data['EmergDurat']+.1,.15,'UnMethalated Vrn1', rotation = ang, verticalalignment='bottom',horizontalalignment = 'center',color='deepskyblue')
    
    plt.text(5,0.92,'MT',color='k',horizontalalignment = 'right')
    
def Vrn2Plots(data, ax, c):
    lowestTS = 16
    highestTS = 0
    pos = 0

    plt.ylabel('Apparent Vrn Expression')
    plt.xlabel('Haun Stage')
    plt.ylim(0,2.5)
    plt.xlim(-2,18)
    plt.text(17,0.05,c,horizontalalignment='right')
    
    plt.text(-1, 1.05,r'$VI$',horizontalalignment='center',verticalalignment='center')
    plt.plot([-2,20],[1,1],ls='dotted',color='k')

    plt.plot([data['VI_WL']],[1],'o',mec='r',mfc='r',ms=10)
    plt.text(data['VI_WL'],.67,'WL',horizontalalignment='center',verticalalignment='bottom')
    plt.plot([data['VI_WL']],[0.9],'^',color = 'k',ms=7)
    plt.plot([data['VI_WL']]*2,[0.9,0.8],'-',color = 'k',ms=7)
 
    #Extrapolate BaseVrn1 back to MinVI to show how much Vrn1 was required to overcome Vrn2
    plt.plot([0,data['DRVrn2HSWL']],[data['rVrn2'],0],'--',color='g')
    plt.text(-.5,min(2.15,data['rVrn2'])+.02,R'$rVrn2$',color='crimson')
    plt.plot([-.5,2.5],[data['rVrn2']]*2, '-', color='crimson')
    

    # #Plot max Vrn1 ER
    maxpos = 0.5
    MidP = (data['DRVrn2HSWL']* maxpos)
    if data['DRVrn2HSWL']!=0:
        ang = np.arctan(data['rVrn2']/(data['DRVrn2HSWL']))*(180/np.pi)
    else:
        ang = 0
    arc = patches.Arc([MidP,data['rVrn2']*(1-maxpos)],1.0,.125,angle=0.0,theta1=360-ang,theta2=0,linewidth=2, fill=False, color='g')
    ax.add_patch(arc)
    plt.plot([MidP,MidP+1],[data['rVrn2']*(1-maxpos)]*2,'-',color='g')
    plt.text(MidP+.6,data['rVrn2']*(1-maxpos)-0.08,r'$-(rVrnB_{Veg})$',color='g',)

    plt.plot([data['DRVrn2HSWL'],data['VI_WL']],[0,data['URVrn3HSWL']*(data['rVrnBVeg'] * data['rVrn3Veg'])],'--',color='grey')
    maxpos = 0.5
    MidP = data['DRVrn2HSWL']+(data['VI_WL']-data['DRVrn2HSWL'])* maxpos
    ang = np.arctan(1/(data['VI_WL']-data['DRVrn2HSWL']))*(180/np.pi)
    arc = patches.Arc([MidP,1*maxpos],1.0,.125,angle=0.0,theta1=0,theta2=ang,linewidth=2, fill=False, color='grey')
    ax.add_patch(arc)
    plt.plot([MidP,MidP+1],[maxpos]*2,'-',color='grey')
    plt.text(MidP+.6,maxpos+0.03,r'$rVrnB_{Veg} * rVrn3_{Veg}$',color='grey',)
    plt.plot([data['VI_WL']]*2, [1.0,1.15],':',color='k')
    plt.plot([data['DRVrn2HSWL']+.1,data['DRVrn2HSWL']+data['URVrn3HSWL']],[1.15,1.15],'>-.',color='k')
    plt.text(data['DRVrn2HSWL']+data['URVrn3HSWL']/2,1.15,r'$URVrn3^{HS}_{WL}$',horizontalalignment='center',verticalalignment='bottom')
    
    plt.plot(0,0.03,'v',mec = 'k',mfc='white',ms=7)
    plt.plot([0,0,np.nan,0,0,np.nan,0,0],[.06,.1,np.nan,0.45,data['rVrn2'],np.nan,data['rVrn2']+.1,1.],'-',color = 'k',ms=7)
    plt.text(0,.15,'Emerg', rotation = 90, verticalalignment='bottom',horizontalalignment = 'center')
    
    plt.plot([0+.1,data['DRVrn2HSWL']],[1.15,1.15],'>-.',color='k')
    plt.plot([0,0,np.nan,data['DRVrn2HSWL'],data['DRVrn2HSWL']],[1,1.15,np.nan,1.15,0],':',color='k')
    plt.text(data['DRVrn2HSWL']/2,1.15,r'$DRVrn2^{HS}_{WL}$',horizontalalignment='center',verticalalignment='bottom')      

Overwriting C:\Users\Cflhxb\AppData\Local\anaconda3\Lib\CAMP.py
